In [ ]:
#lane detection and control using open cv
#accesing the csi camera inputs to operate using opencv
#jetson hacks

In [ ]:
#initialisation
import cv2
from jetracer.nvidia_racecar import NvidiaRacecar
from jetcam.csi_camera import CSICamera


In [ ]:
#edge detection & Thresholding
def CannyEdge(image):
  gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
  blur = cv2.GaussianBlur(gray, (5,5), 0)
  cannyImage = cv2.Canny(blur, 50, 150)
  return cannyImage

In [ ]:
#ROI
def region_of_interest(image):
    height = image.shape[0]
    triangle = np.array([[(200, height),(550, 250),(1100, height),]], np.int32)
    mask = np.zeros_like(image)
    cv2.fillPoly(mask, triangle, 255)
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

In [ ]:
#display of lane line detected
def display_lines(image, lines):
    line_image = np.zeros_like(image)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line.reshape(4)
            cv2.line(line_image, (x1, y1), (x2, y2), (255, 0, 0), 15)
    return line_image


In [ ]:
#making of points
def make_points(image, line_parameters,lane):
    if line_parameters.shape != (2,):
        slope, intercept = lane;
        print(slope);
        y1 = int(image.shape[0]);
        y2 = int(y1*3/5);
        x1 = int((y1 - intercept)/slope);
        x2 = int((y2 - intercept)/slope);
        return [[x1, y1, x2, y2]];
    else:
        slope, intercept = line_parameters;
        print(slope);
        y1 = int(image.shape[0])
        y2 = int(y1*3/5)
        x1 = int((y1 - intercept)/slope)
        x2 = int((y2 - intercept)/slope)
        return [[x1, y1, x2, y2]]

In [ ]:
#average slope intercept
def average_slope_intercept(image, lines):
    left_fit = []
    right_fit = []
    if lines is None:
        return None
    for line in lines:
        x1, y1, x2, y2 = line.reshape(4);
        print(x1,y1,x2,y2);
        parameters = np.polyfit((x1, x2), (y1, y2), 1);
        slope = parameters[0];
        intercept = parameters[1];
        if slope < 0:
            left_fit.append((slope, intercept));
            left.append((slope,intercept));
        else:
            right_fit.append((slope, intercept));
            right.append((slope,intercept));
    left_fit_average = np.average(left_fit, axis = 0);
    right_fit_average = np.average(right_fit, axis = 0);
    a=np.average(left,axis=0);
    b=np.average(right,axis=0);
    left_line = make_points(image, left_fit_average,a);
    right_line = make_points(image, right_fit_average,b);
    return np.array((left_line, right_line));

In [ ]:
#image processing for lane following , i.e . setting points for calculation for pid steering control

In [ ]:
#pid control

In [ ]:
#getting input real-time from the csi camera



In [ ]:
#lane detection pipeline
canny = CannyEdge(frame)
cropped_Image = region_of_interest(canny)
rho = 2
theta = np.pi/180
threshold = 100
lines = cv2.HoughLinesP(cropped_Image,rho, theta, threshold, np.array ([ ]), minLineLength=40, maxLineGap=5)
averaged_lines = average_slope_intercept(frame, lines)
line_image = display_lines(frame, averaged_lines)
combo_image = cv2.addWeighted(frame, 0.8, line_image, 1, 1)
cv2.imshow("Image", combo_image)
if cv2.waitKey(1) & 0xFF == ord('q'):
    break
cap.release()
cv2.destroyAllWindows()

In [ ]:
#control pipeline